In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from pyproj import transform, CRS, Proj
import json
from dotenv import find_dotenv, load_dotenv
from pathlib import Path
from os import environ
from shapely.geometry import Point, Polygon
from shapely.ops import transform as shapely_transform
from functools import partial

# Custom modules
from app import GeocoderUtilities
from app.utils import connection

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
env_loc = find_dotenv('.env')
load_dotenv(env_loc)

True

In [7]:
cur = connection.cursor()

In [8]:
res = cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
for i in res:
    print(i['name'])

UK_Wards
UK_Local_Authorities
UK_Constituencies
UK_Election_2019
Postcode_coordinates
UK_Wards_Population
local_authority_population_projections
property_prices_2019


In [9]:
geo = GeocoderUtilities()
google_key = environ.get('GOOGLE_KEY')

In [10]:
## Import ward shapefile and merge with population data
ward_df = geo.sql_to_df('UK_Wards_Population', cur)
# ward_shp = gpd.read_file('app/data/shapefiles/Wards__December_2017__Boundaries_in_GB.shx').to_crs(epsg=4326)
# wards_gdf = ward_df.merge(ward_shp, left_on='Code', right_on='wd17cd', how='left')

In [11]:
sales_df = geo.sql_to_df('property_prices_2019', cur)
sales_gdf = gpd.GeoDataFrame(sales_df, geometry=gpd.points_from_xy(sales_df.longitude,
                                                                   sales_df.latitude)).set_crs(epsg=4326)

In [16]:
df = pd.DataFrame({'location':['N4 2AR',np.nan]})
df

,location
0,N4 2AR
1,NaN


In [1]:
def df_merge_with_sales(dataframe: object, address_column: str, api_key: str, 
                        buffer_kilometres: float, region: str = 'uk',
                       agg_function_list: str = 'sum,mean,median,count'):
    
    buffer_df = self.df_to_geodf_with_buffer(dataframe, address_column, api_key, buffer_kilometres, region)
    sales_in_buffer = gpd.sjoin(buffer_df, sales_gdf, how='left', op='intersects')
    full_gdf = sales_in_buffer\
               .groupby('location')\
               .agg({'AMOUNT':agg_function_list.split(',')})\
               .reset_index()\
               .dropna()
    
    return full_gdf

In [3]:
## Process
## 1. List of postcodes to analyse - x
## 2. Geocode each postcode, returning lat-lng (func: geocode) -x
## 3. Generate buffer boundary overlay (func: buffer) -x
## 4a. Select wards within buffer (func: overlay boundaries) -x
## 4b. Return dataframe of wards, summarise stats -x
## 5a. Select properties within buffer (func: overlay points) -x
## 5b. Return dataframe of properties, summarise stats

## Instantiated
## - Merging population dataframe with shapefile into GeoDataFrame -x
## - GeoDataFrame with price, type, size, and Point Geometry -x